# Distributed training

<div align="left">
<a target="_blank" href="https://console.anyscale.com/"><img src="https://raw.githubusercontent.com/ray-project/ray/c34b74c22a9390aa89baf80815ede59397786d2e/doc/source/_static/img/run-on-anyscale.svg"></a>&nbsp;

<a href="https://github.com/anyscale/multimodal-ai" role="button"><img src="https://img.shields.io/static/v1?label=&amp;message=View%20On%20GitHub&amp;color=586069&amp;logo=github&amp;labelColor=2f363d"></a>&nbsp;
</div>

This tutorial executes a distributed training workload that connects the following heterogeneous workloads:
- preprocess the dataset prior to training
- distributed training with Ray Train and PyTorch with observability
- evaluation (batch inference and eval logic)
- save model artifacts to a model registry (MLOps)

**Note**: this tutorial doesn't tune the model but see [Ray Tune](https://docs.ray.io/en/latest/tune/index.html) for experiment execution and hyperparameter tuning at any scale.

<img src="https://raw.githubusercontent.com/anyscale/multimodal-ai/refs/heads/main/images/distributed_training.png" width=1000>

In [ ]:
%%bash
pip install -q -r /home/ray/default/requirements.txt
pip install -q -e /home/ray/default/doggos


**Note**: A kernel restart may be required for all dependencies to become available.

In [ ]:
%load_ext autoreload
%autoreload all


In [ ]:
import os
import ray
import sys
sys.path.append(os.path.abspath("../doggos/"))


In [ ]:
# If using UV
# os.environ.pop("RAY_RUNTIME_ENV_HOOK", None)


'cgroup_runtime_plugin._hook'

In [ ]:
# Enable Ray Train v2. It's too good to wait for public release!
os.environ["RAY_TRAIN_V2_ENABLED"] = "1"
ray.init(
    # connect to existing ray runtime (from previous notebook if still running)
    address=os.environ.get("RAY_ADDRESS", "auto"),
    runtime_env={
        "env_vars": {"RAY_TRAIN_V2_ENABLED": "1"},
        # "py_executable": "uv run", # if using uv 
        # "working_dir": "/home/ray/default",  # if using uv 
    },
)


2025-08-21 20:05:05,476	INFO worker.py:1747 -- Connecting to existing Ray cluster at address: 10.0.52.10:6379...
2025-08-21 20:05:05,487	INFO worker.py:1918 -- Connected to Ray cluster. View the dashboard at https://session-466hy7cqu1gzrp8zk8l4byz7l7.i.anyscaleuserdata.com 
2025-08-21 20:05:05,534	INFO packaging.py:588 -- Creating a file package for local module '/home/ray/default'.


2025-08-21 20:05:05,599	INFO packaging.py:380 -- Pushing file package 'gcs://_ray_pkg_f5e3d74a1b6904d1.zip' (13.52MiB) to Ray cluster...
2025-08-21 20:05:05,657	INFO packaging.py:393 -- Successfully pushed file package 'gcs://_ray_pkg_f5e3d74a1b6904d1.zip'.


Python version:,3.12.11
Ray version:,2.48.0
Dashboard:,http://session-466hy7cqu1gzrp8zk8l4byz7l7.i.anyscaleuserdata.com


(autoscaler +1m25s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +1m25s) [autoscaler] [48CPU-192GB] Attempting to add 3 nodes to the cluster (increasing from 0 to 3).
(autoscaler +1m25s) [autoscaler] [48CPU-192GB|m5.12xlarge] [us-west-2a] [on-demand] Launched 3 instances.
(autoscaler +2m2s) [autoscaler] Cluster upscaled to {56 CPU, 0 GPU}.
(autoscaler +2m5s) [autoscaler] Cluster upscaled to {104 CPU, 0 GPU}.
(autoscaler +2m25s) [autoscaler] Cluster upscaled to {152 CPU, 0 GPU}.


In [ ]:
%%bash
# This will be removed once Ray Train v2 is enabled by default.
echo "RAY_TRAIN_V2_ENABLED=1" > /home/ray/default/.env


In [ ]:
# Load env vars in notebooks.
from dotenv import load_dotenv
load_dotenv()


True

## Preprocess

You need to convert the classes to labels (unique integers) so that you can train a classifier that can correctly predict the class given an input image. But before you do this, apply the same data ingestion and preprocessing as the previous notebook.

In [ ]:
def add_class(row):
    row["class"] = row["path"].rsplit("/", 3)[-2]
    return row


In [ ]:
# Preprocess data splits.
train_ds = ray.data.read_images("s3://doggos-dataset/train", include_paths=True, shuffle="files")
train_ds = train_ds.map(add_class)
val_ds = ray.data.read_images("s3://doggos-dataset/val", include_paths=True)
val_ds = val_ds.map(add_class)


Define a `Preprocessor` class that:
- creates an embedding. A later step moves the embedding layer outside of the model since you freeze the embedding layer's weights and so you don't have to do it repeatedly as part of the model's forward pass, saving on unnecessary compute.
- convert the classes into labels for the classifier. 

While you could've just done this step as a simple operation, you're taking the time to organize it as a class so that you can save and load for inference later.

In [ ]:
def convert_to_label(row, class_to_label):
    if "class" in row:
        row["label"] = class_to_label[row["class"]]
    return row


In [ ]:
import numpy as np
from PIL import Image
import torch
from transformers import CLIPModel, CLIPProcessor
from doggos.embed import EmbedImages


In [ ]:
class Preprocessor:
    """Preprocessor class."""
    def __init__(self, class_to_label=None):
        self.class_to_label = class_to_label or {}  # mutable defaults
        self.label_to_class = {v: k for k, v in self.class_to_label.items()}
        
    def fit(self, ds, column):
        self.classes = ds.unique(column=column)
        self.class_to_label = {tag: i for i, tag in enumerate(self.classes)}
        self.label_to_class = {v: k for k, v in self.class_to_label.items()}
        return self
    
    def transform(self, ds, concurrency=4, batch_size=64, num_gpus=1):
        ds = ds.map(
            convert_to_label, 
            fn_kwargs={"class_to_label": self.class_to_label},
        )
        ds = ds.map_batches(
            EmbedImages,
            fn_constructor_kwargs={
                "model_id": "openai/clip-vit-base-patch32", 
                "device": "cuda",
            },
            concurrency=4,
            batch_size=64,
            num_gpus=1,
            accelerator_type="L4",
        )
        ds = ds.drop_columns(["image"])
        return ds

    def save(self, fp):
        with open(fp, "w") as f:
            json.dump(self.class_to_label, f)


In [ ]:
# Preprocess.
preprocessor = Preprocessor()
preprocessor = preprocessor.fit(train_ds, column="class")
train_ds = preprocessor.transform(ds=train_ds)
val_ds = preprocessor.transform(ds=val_ds)


2025-08-21 20:05:22,045	INFO dataset.py:3057 -- Tip: Use `take_batch()` instead of `take() / show()` to return records in pandas or numpy batch format.
2025-08-21 20:05:22,047	INFO logging.py:295 -- Registered dataset logger for dataset dataset_30_0
2025-08-21 20:05:22,069	INFO streaming_executor.py:117 -- Starting execution of Dataset dataset_30_0. Full logs are in /tmp/ray/session_2025-08-21_18-48-13_464408_2298/logs/ray-data
2025-08-21 20:05:22,070	INFO streaming_executor.py:118 -- Execution plan of Dataset dataset_30_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[Map(add_class)] -> AllToAllOperator[Aggregate] -> LimitOperator[limit=1]


Running 0: 0.00 row [00:00, ? row/s]

- ListFiles 1: 0.00 row [00:00, ? row/s]

- ReadFiles 2: 0.00 row [00:00, ? row/s]

- Map(add_class) 3: 0.00 row [00:00, ? row/s]

...

Shuffle Map 6:   0%|          | 0.00/1.00 [00:00<?, ? row/s]

Shuffle Reduce 7:   0%|          | 0.00/1.00 [00:00<?, ? row/s]

- limit=1 8: 0.00 row [00:00, ? row/s]

2025-08-21 20:05:22,149	WARNING resource_manager.py:130 -- ⚠️  Ray's object store is configured to use only 27.2% of available memory (9.4GB out of 34.4GB total). For optimal Ray Data performance, we recommend setting the object store to at least 50% of available memory. You can do this by setting the 'object_store_memory' parameter when calling ray.init() or by setting the RAY_DEFAULT_OBJECT_STORE_MEMORY_PROPORTION environment variable.
2025-08-21 20:06:26,797	INFO streaming_executor.py:231 -- ✔️  Dataset dataset_30_0 execution finished in 64.72 seconds


(raylet) Using CPython 3.12.11 interpreter at: /home/ray/anaconda3/bin/python3.12
(raylet) Creating virtual environment at: .venv
(raylet)    Building doggos @ file:///tmp/ray/session_2025-08-21_18-48-13_464408_2298/runtime_resources/working_dir_files/_ray_pkg_f5e3d74a1b6904d1/doggos
(raylet)       Built doggos @ file:///tmp/ray/session_2025-08-21_18-48-13_464408_2298/runtime_resources/working_dir_files/_ray_pkg_f5e3d74a1b6904d1/doggos
(raylet) warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
(raylet)          If the cache and target directories are on different filesystems, hardlinking may not be supported.
(raylet)          If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
(raylet) Installed 172 packages in 29.25s


<div class="alert alert-block alert"> <b> Data processing</b> 

See this extensive guide on [data loading and preprocessing](https://docs.ray.io/en/latest/train/user-guides/data-loading-preprocessing.html) for the last-mile preprocessing you need to do prior to training your models. However, Ray Data does support performant joins, filters, aggregations, etc., for the more structure data processing your workloads may need.

In [ ]:
import shutil


In [ ]:
# Write processed data to cloud storage.
preprocessed_data_path = os.path.join("/mnt/cluster_storage", "doggos/preprocessed_data")
if os.path.exists(preprocessed_data_path):  # Clean up.
    shutil.rmtree(preprocessed_data_path)
preprocessed_train_path = os.path.join(preprocessed_data_path, "preprocessed_train")
preprocessed_val_path = os.path.join(preprocessed_data_path, "preprocessed_val")
train_ds.write_parquet(preprocessed_train_path)
val_ds.write_parquet(preprocessed_val_path)


2025-08-21 20:08:26,684	INFO logging.py:295 -- Registered dataset logger for dataset dataset_38_0
2025-08-21 20:08:26,700	INFO streaming_executor.py:117 -- Starting execution of Dataset dataset_38_0. Full logs are in /tmp/ray/session_2025-08-21_18-48-13_464408_2298/logs/ray-data
2025-08-21 20:08:26,701	INFO streaming_executor.py:118 -- Execution plan of Dataset dataset_38_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[Map(add_class)->Map(convert_to_label)] -> ActorPoolMapOperator[MapBatches(EmbedImages)] -> TaskPoolMapOperator[MapBatches(drop_columns)->Write]


Running 0: 0.00 row [00:00, ? row/s]

- ListFiles 1: 0.00 row [00:00, ? row/s]

- ReadFiles 2: 0.00 row [00:00, ? row/s]

- Map(add_class)->Map(convert_to_label) 3: 0.00 row [00:00, ? row/s]

...

- ReadFiles 2: 0.00 row [00:00, ? row/s]

- Map(add_class)->Map(convert_to_label) 3: 0.00 row [00:00, ? row/s]

- MapBatches(EmbedImages) 4: 0.00 row [00:00, ? row/s]

- MapBatches(drop_columns)->Write 5: 0.00 row [00:00, ? row/s]

2025-08-21 20:11:10,014	WARNING streaming_executor_state.py:764 -- Operator produced a RefBundle with a different schema than the previous one. Previous schema: image: extension<ray.data.arrow_variable_shaped_tensor<ArrowVariableShapedTensorType>>
path: string, new schema: image: extension<ray.data.arrow_tensor_v2<ArrowTensorTypeV2>>
path: string. This may lead to unexpected behavior.
2025-08-21 20:11:10,440	WARNING streaming_executor_state.py:764 -- Operator produced a RefBundle with a different schema than the previous one. Previous schema: image: extension<ray.data.arrow_variable_shaped_tensor<ArrowVariableShapedTensorType>>
path: string
class: string
label: int64, new schema: image: extension<ray.data.arrow_tensor_v2<ArrowTensorTypeV2>>
path: string
class: string
label: int64. This may lead to unexpected behavior.
2025-08-21 20:11:19,843	INFO streaming_executor.py:231 -- ✔️  Dataset dataset_41_0 execution finished in 11.66 seconds
2025-08-21 20:11:19,886	INFO dataset.py:4621 -- Dat

<div class="alert alert-block alert"> <b> Store often, save compute</b> 

Store the preprocessed data into shared cloud storage to:
- save a record of what this preprocessed data looks like
- avoid triggering the entire preprocessing for each batch the model processes
- avoid [`materialize`](https://docs.ray.io/en/latest/data/api/doc/ray.data.Dataset.materialize.html) of the preprocessed data because you shouldn't force large data to fit in memory

## Model

Define the model -- a simple two layer neural net with Softmax layer to predict class probabilities. Notice that it's all just base PyTorch and nothing else.

In [ ]:
import json
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F


In [ ]:
class ClassificationModel(torch.nn.Module):
    def __init__(self, embedding_dim, hidden_dim, dropout_p, num_classes):
        super().__init__()
        # Hyperparameters
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.dropout_p = dropout_p
        self.num_classes = num_classes

        # Define layers
        self.fc1 = nn.Linear(embedding_dim, hidden_dim)
        self.batch_norm = nn.BatchNorm1d(hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_p)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, batch):
        z = self.fc1(batch["embedding"])
        z = self.batch_norm(z)
        z = self.relu(z)
        z = self.dropout(z)
        z = self.fc2(z)
        return z

    @torch.inference_mode()
    def predict(self, batch):
        z = self(batch)
        y_pred = torch.argmax(z, dim=1).cpu().numpy()
        return y_pred

    @torch.inference_mode()
    def predict_probabilities(self, batch):
        z = self(batch)
        y_probs = F.softmax(z, dim=1).cpu().numpy()
        return y_probs

    def save(self, dp):
        Path(dp).mkdir(parents=True, exist_ok=True)
        with open(Path(dp, "args.json"), "w") as fp:
            json.dump({
                "embedding_dim": self.embedding_dim,
                "hidden_dim": self.hidden_dim,
                "dropout_p": self.dropout_p,
                "num_classes": self.num_classes,
            }, fp, indent=4)
        torch.save(self.state_dict(), Path(dp, "model.pt"))

    @classmethod
    def load(cls, args_fp, state_dict_fp, device="cpu"):
        with open(args_fp, "r") as fp:
            model = cls(**json.load(fp))
        model.load_state_dict(torch.load(state_dict_fp, map_location=device))
        return model


In [ ]:
# Initialize model.
num_classes = len(preprocessor.classes)
model = ClassificationModel(
    embedding_dim=512, 
    hidden_dim=256, 
    dropout_p=0.3, 
    num_classes=num_classes,
)
print (model)


ClassificationModel(
  (fc1): Linear(in_features=512, out_features=256, bias=True)
  (batch_norm): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=256, out_features=36, bias=True)
)


## Batching

Take a look at a sample batch of data and ensure that tensors have the proper data type.

In [ ]:
from ray.train.torch import get_device


In [ ]:
def collate_fn(batch):
    dtypes = {"embedding": torch.float32, "label": torch.int64}
    tensor_batch = {}
    for key in dtypes.keys():
        if key in batch:
            tensor_batch[key] = torch.as_tensor(
                batch[key],
                dtype=dtypes[key],
                device=get_device(),
            )
    return tensor_batch


In [ ]:
# Sample batch
sample_batch = train_ds.take_batch(batch_size=3)
collate_fn(batch=sample_batch)


2025-08-21 20:11:20,682	INFO logging.py:295 -- Registered dataset logger for dataset dataset_43_0
2025-08-21 20:11:20,693	INFO streaming_executor.py:117 -- Starting execution of Dataset dataset_43_0. Full logs are in /tmp/ray/session_2025-08-21_18-48-13_464408_2298/logs/ray-data
2025-08-21 20:11:20,693	INFO streaming_executor.py:118 -- Execution plan of Dataset dataset_43_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[Map(add_class)->Map(convert_to_label)] -> ActorPoolMapOperator[MapBatches(EmbedImages)] -> TaskPoolMapOperator[MapBatches(drop_columns)] -> LimitOperator[limit=3]


Running 0: 0.00 row [00:00, ? row/s]

- ListFiles 1: 0.00 row [00:00, ? row/s]

- ReadFiles 2: 0.00 row [00:00, ? row/s]

- Map(add_class)->Map(convert_to_label) 3: 0.00 row [00:00, ? row/s]

- MapBatches(EmbedImages) 4: 0.00 row [00:00, ? row/s]

- MapBatches(drop_columns) 5: 0.00 row [00:00, ? row/s]

- limit=3 6: 0.00 row [00:00, ? row/s]

2025-08-21 20:11:29,549	INFO streaming_executor.py:231 -- ✔️  Dataset dataset_43_0 execution finished in 8.85 seconds
/tmp/ipykernel_44250/417303983.py:6: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:203.)
  tensor_batch[key] = torch.as_tensor(


{'embedding': tensor([[ 0.3172,  0.1145, -0.1010,  ...,  0.0425,  0.3932,  0.3251],
         [ 0.6690,  0.1264, -0.3607,  ...,  0.7516,  0.3557,  0.2929],
         [-0.2309,  0.1717, -0.2901,  ...,  0.8024,  0.3071, -0.0147]]),
 'label': tensor([34, 29, 35])}

## Model registry

Create a model registry in [Anyscale user storage](https://docs.anyscale.com/configuration/storage/#user-storage) to save the model checkpoints to. Use OSS MLflow but you can easily [set up other experiment trackers](https://docs.ray.io/en/latest/train/user-guides/experiment-tracking.html) with Ray.

In [ ]:
import shutil


In [ ]:
model_registry = "/mnt/cluster_storage/mlflow/doggos"
if os.path.isdir(model_registry):
    shutil.rmtree(model_registry)  # clean up
os.makedirs(model_registry, exist_ok=True)


## Training

Define the training workload by specifying the:
- experiment and model parameters
- compute scaling configuration
- forward pass for batches of training and validation data
- train loop for each epoch of data and checkpointing

<img src="https://raw.githubusercontent.com/anyscale/multimodal-ai/refs/heads/main/images/trainer.png" width=700>

In [ ]:
# Train loop config.
experiment_name = "doggos"
train_loop_config = {
    "model_registry": model_registry,
    "experiment_name": experiment_name,
    "embedding_dim": 512,
    "hidden_dim": 256,
    "dropout_p": 0.3,
    "lr": 1e-3,
    "lr_factor": 0.8,
    "lr_patience": 3,
    "num_epochs": 20,
    "batch_size": 256,
}


In [ ]:
# Scaling config
num_workers = 4
scaling_config = ray.train.ScalingConfig(
    num_workers=num_workers,
    use_gpu=True,
    resources_per_worker={"CPU": 8, "GPU": 2},
    accelerator_type="L4",
)


In [ ]:
import tempfile
import mlflow
import numpy as np
from ray.train.torch import TorchTrainer


In [ ]:
def train_epoch(ds, batch_size, model, num_classes, loss_fn, optimizer):
    model.train()
    loss = 0.0
    ds_generator = ds.iter_torch_batches(batch_size=batch_size, collate_fn=collate_fn)
    for i, batch in enumerate(ds_generator):
        optimizer.zero_grad()  # Reset gradients.
        z = model(batch)  # Forward pass.
        targets = F.one_hot(batch["label"], num_classes=num_classes).float()
        J = loss_fn(z, targets)  # Define loss.
        J.backward()  # Backward pass.
        optimizer.step()  # Update weights.
        loss += (J.detach().item() - loss) / (i + 1)  # Cumulative loss
    return loss


In [ ]:
def eval_epoch(ds, batch_size, model, num_classes, loss_fn):
    model.eval()
    loss = 0.0
    y_trues, y_preds = [], []
    ds_generator = ds.iter_torch_batches(batch_size=batch_size, collate_fn=collate_fn)
    with torch.inference_mode():
        for i, batch in enumerate(ds_generator):
            z = model(batch)
            targets = F.one_hot(batch["label"], num_classes=num_classes).float()  # one-hot (for loss_fn)
            J = loss_fn(z, targets).item()
            loss += (J - loss) / (i + 1)
            y_trues.extend(batch["label"].cpu().numpy())
            y_preds.extend(torch.argmax(z, dim=1).cpu().numpy())
    return loss, np.vstack(y_trues), np.vstack(y_preds)


In [ ]:
def train_loop_per_worker(config):
    # Hyperparameters.
    model_registry = config["model_registry"]
    experiment_name = config["experiment_name"]
    embedding_dim = config["embedding_dim"]
    hidden_dim = config["hidden_dim"]
    dropout_p = config["dropout_p"]
    lr = config["lr"]
    lr_factor = config["lr_factor"]
    lr_patience = config["lr_patience"]
    num_epochs = config["num_epochs"]
    batch_size = config["batch_size"]
    num_classes = config["num_classes"]

    # Experiment tracking.
    if ray.train.get_context().get_world_rank() == 0:
        mlflow.set_tracking_uri(f"file:{model_registry}")
        mlflow.set_experiment(experiment_name)
        mlflow.start_run()
        mlflow.log_params(config)

    # Datasets.
    train_ds = ray.train.get_dataset_shard("train")
    val_ds = ray.train.get_dataset_shard("val")

    # Model.
    model = ClassificationModel(
        embedding_dim=embedding_dim, 
        hidden_dim=hidden_dim, 
        dropout_p=dropout_p, 
        num_classes=num_classes,
    )
    model = ray.train.torch.prepare_model(model)

    # Training components.
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, 
        mode="min", 
        factor=lr_factor, 
        patience=lr_patience,
    )

    # Training.
    best_val_loss = float("inf")
    for epoch in range(num_epochs):
        # Steps
        train_loss = train_epoch(train_ds, batch_size, model, num_classes, loss_fn, optimizer)
        val_loss, _, _ = eval_epoch(val_ds, batch_size, model, num_classes, loss_fn)
        scheduler.step(val_loss)

        # Checkpoint (metrics, preprocessor and model artifacts).
        with tempfile.TemporaryDirectory() as dp:
            model.module.save(dp=dp)
            metrics = dict(lr=optimizer.param_groups[0]["lr"], train_loss=train_loss, val_loss=val_loss)
            with open(os.path.join(dp, "class_to_label.json"), "w") as fp:
                json.dump(config["class_to_label"], fp, indent=4)
            if ray.train.get_context().get_world_rank() == 0:  # only on main worker 0
                mlflow.log_metrics(metrics, step=epoch)
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    mlflow.log_artifacts(dp)

    # End experiment tracking.
    if ray.train.get_context().get_world_rank() == 0:
        mlflow.end_run()


<div class="alert alert-block alert"> <b> Minimal change to your training code</b> 

Notice that there isn't much new Ray Train code on top of the base PyTorch code. You specified how you want to scale out the training workload, load the Ray datasets, and then checkpoint on the main worker node and that's it. See these guides ([PyTorch](https://docs.ray.io/en/latest/train/getting-started-pytorch.html), [PyTorch Lightning](https://docs.ray.io/en/latest/train/getting-started-pytorch-lightning.html), [Hugging Face Transformers](https://docs.ray.io/en/latest/train/getting-started-transformers.html)) to see the minimal change in code needed to distribute your training workloads. See this extensive list of [Ray Train user guides](https://docs.ray.io/en/latest/train/user-guides.html).

In [ ]:
# Load preprocessed datasets.
preprocessed_train_ds = ray.data.read_parquet(preprocessed_train_path)
preprocessed_val_ds = ray.data.read_parquet(preprocessed_val_path)


In [ ]:
# Trainer.
train_loop_config["class_to_label"] = preprocessor.class_to_label
train_loop_config["num_classes"] = len(preprocessor.class_to_label)
trainer = TorchTrainer(
    train_loop_per_worker=train_loop_per_worker,
    train_loop_config=train_loop_config,
    scaling_config=scaling_config,
    datasets={"train": preprocessed_train_ds, "val": preprocessed_val_ds},
)


In [ ]:
# Train.
results = trainer.fit()


(autoscaler +6m40s) [autoscaler] [4xL4:48CPU-192GB] Attempting to add 2 nodes to the cluster (increasing from 0 to 2).
(autoscaler +6m40s) [autoscaler] [4xL4:48CPU-192GB|g6.12xlarge] [us-west-2c] [on-demand] Launched 2 instances.


(TrainController pid=54671) Retrying the launch of the training worker group. The previous launch attempt encountered the following failure:
(TrainController pid=54671) The worker group startup timed out after 30.0 seconds waiting for 4 workers. Potential causes include: (1) temporary insufficient cluster resources while waiting for autoscaling (ignore this warning in this case), (2) infeasible resource request where the provided `ScalingConfig` cannot be satisfied), and (3) transient network issues. Set the RAY_TRAIN_WORKER_GROUP_START_TIMEOUT_S environment variable to increase the timeout.
(TrainController pid=54671) [State Transition] SCHEDULING -> RESCHEDULING.
(TrainController pid=54671) [State Transition] RESCHEDULING -> SCHEDULING.
(TrainController pid=54671) Attempting to start training worker group of size 4 with the following resources: [{'CPU': 8, 'GPU': 2, 'accelerator_type:L4': 0.001}] * 4
(raylet, ip=10.0.164.18) Using CPython 3.12.11 interpreter at: /home/ray/anaconda3/b

(autoscaler +7m25s) [autoscaler] Cluster upscaled to {408 CPU, 12 GPU}.


(raylet, ip=10.0.164.18)  Downloading numpy
(raylet, ip=10.0.164.18)  Downloading sqlalchemy
(raylet, ip=10.0.164.18)  Downloading widgetsnbextension
(raylet, ip=10.0.164.18)  Downloading kiwisolver
(raylet, ip=10.0.164.18)  Downloading pillow
(raylet, ip=10.0.164.18)  Downloading debugpy
(raylet, ip=10.0.164.18)  Downloading virtualenv
(raylet, ip=10.0.164.18)  Downloading nvidia-cuda-nvrtc-cu12 [repeated 15x across cluster]
(raylet, ip=10.0.164.18)  Downloading pandas
(raylet, ip=10.0.164.18)  Downloading hf-xet
(raylet, ip=10.0.164.18)  Downloading setuptools
(raylet, ip=10.0.164.18)  Downloading uvloop
(raylet, ip=10.0.164.18)  Downloading pygments
(raylet, ip=10.0.164.18)  Downloading fonttools
(raylet, ip=10.0.164.18)  Downloading py-spy
(raylet, ip=10.0.164.18)  Downloading sympy
(raylet, ip=10.0.164.18)  Downloading grpcio
(raylet, ip=10.0.164.18)  Downloading polars
(raylet, ip=10.0.164.18)  Downloading botocore
(raylet, ip=10.0.164.18)  Downloading aiohttp
(raylet, ip=10.0.16

(pid=55351) Running 0: 0.00 row [00:00, ? row/s]

...

(pid=55351) - split(4, equal=True) 3: 0.00 row [00:00, ? row/s]

(pid=55396) Running 0: 0.00 row [00:00, ? row/s]

(pid=55396) - ListFiles 1: 0.00 row [00:00, ? row/s]

(pid=55396) - ReadFiles 2: 0.00 row [00:00, ? row/s]

(pid=55396) - split(4, equal=True) 3: 0.00 row [00:00, ? row/s]

## Ray Train

- automatically handles **multi-node, multi-GPU** setup with no manual SSH setup or hostfile configs. 
- define **per-worker fractional resource requirements**, for example, 2 CPUs and 0.5 GPU per worker.
- run on **heterogeneous machines** and scale flexibly, for example, CPU for preprocessing and GPU for training. 
- built-in **fault tolerance** with retry of failed workers and continue from last checkpoint.
- supports Data Parallel, Model Parallel, Parameter Server, and even custom strategies.
- [Ray Compiled graphs](https://docs.ray.io/en/latest/ray-core/compiled-graph/ray-compiled-graph.html) allow you to even define different parallelism for jointly optimizing multiple models like Megatron, DeepSpeed, etc., or only allow for one global setting.
- You can also use Torch DDP, FSPD, DeepSpeed, etc., under the hood.

🔥 [RayTurbo Train](https://docs.anyscale.com/rayturbo/rayturbo-train) offers even more improvement to the price-performance ratio, performance monitoring and more:
- **elastic training** to scale to a dynamic number of workers, continue training on fewer resources, even on spot instances.
- **purpose-built dashboard** designed to streamline the debugging of Ray Train workloads:
    - Monitoring: View the status of training runs and train workers.
    - Metrics: See insights on training throughput and training system operation time.
    - Profiling: Investigate bottlenecks, hangs, or errors from individual training worker processes.

<img src="https://raw.githubusercontent.com/anyscale/multimodal-ai/refs/heads/main/images/train_dashboard.png" width=700>

You can view experiment metrics and model artifacts in the model registry. You're using OSS MLflow so you can run the server by pointing to the model registry location:

```bash
mlflow server -h 0.0.0.0 -p 8080 --backend-store-uri /mnt/cluster_storage/mlflow/doggos
```

You can view the dashboard by going to the **Overview tab** > **Open Ports**. 

<img src="https://raw.githubusercontent.com/anyscale/multimodal-ai/refs/heads/main/images/mlflow.png" width=1000>

You also have the preceding Ray Dashboard and Train workload specific dashboards.

<img src="https://raw.githubusercontent.com/anyscale/multimodal-ai/refs/heads/main/images/train_metrics.png" width=1000>


In [ ]:
# Sorted runs
mlflow.set_tracking_uri(f"file:{model_registry}")
sorted_runs = mlflow.search_runs(
    experiment_names=[experiment_name], 
    order_by=["metrics.val_loss ASC"])
best_run = sorted_runs.iloc[0]
best_run


run_id                                      635073afa2634a9fa93ae15a04605965
experiment_id                                             505321604523120474
status                                                              FINISHED
artifact_uri               file:///mnt/cluster_storage/mlflow/doggos/5053...
start_time                                  2025-08-12 22:30:11.088000+00:00
end_time                                    2025-08-12 22:30:30.239000+00:00
metrics.lr                                                             0.001
metrics.train_loss                                                  0.389762
metrics.val_loss                                                    0.646278
params.hidden_dim                                                        256
params.num_epochs                                                         20
params.lr_patience                                                         3
params.class_to_label      {'cocker_spaniel': 0, 'bull_mastiff': 1, 'samo...

(autoscaler +9m30s) [autoscaler] Downscaling node i-0758ca207affbe64e (node IP: 10.0.80.63) due to node idle termination.
(autoscaler +9m30s) [autoscaler] Downscaling node i-0458d8cdec562fe7a (node IP: 10.0.84.22) due to node idle termination.
(autoscaler +9m30s) [autoscaler] Downscaling node i-067e3debcee38b433 (node IP: 10.0.101.0) due to node idle termination.
(autoscaler +9m30s) [autoscaler] Downscaling node i-073dfc0bb07bce43c (node IP: 10.0.117.166) due to node idle termination.


run_id                                      861d38c5540f4628bbc10556ffce5f3d
experiment_id                                             798639802550679506
status                                                              FINISHED
artifact_uri               file:///mnt/cluster_storage/mlflow/doggos/7986...
start_time                                  2025-08-21 20:13:30.681000+00:00
end_time                                    2025-08-21 20:13:58.288000+00:00
metrics.val_loss                                                    0.700778
metrics.train_loss                                                  0.368469
metrics.lr                                                             0.001
params.embedding_dim                                                     512
params.lr_factor                                                         0.8
params.num_epochs                                                         20
params.lr_patience                                                         3

## Production Job

You can easily wrap the training workload as a production grade [Anyscale Job](https://docs.anyscale.com/platform/jobs/) ([API ref](https://docs.anyscale.com/reference/job-api/)).

**Note**: 
- This Job uses a `containerfile` to define dependencies, but you could easily use a pre-built image as well.
- You can specify the compute as a [compute config](https://docs.anyscale.com/configuration/compute-configuration/) or inline in a [job config](https://docs.anyscale.com/reference/job-api#job-cli) file.
- When you don't specify compute while launching from a workspace, this configuration defaults to the compute configuration of the workspace.

In [ ]:
%%bash
# Production model training job
anyscale job submit -f /home/ray/default/configs/train_model.yaml


Output
(anyscale +1.0s) Submitting job with config JobConfig(name='train-image-model', image_uri='anyscale/ray:2.48.0-slim-py312-cu128', compute_config=None, env_vars=None, py_modules=None, py_executable=None, cloud=None, project=None, ray_version=None, job_queue_config=None).
(anyscale +3.1s) Uploading local dir '/home/ray/default' to cloud storage.
(anyscale +6.4s) Job 'train-image-model' submitted, ID: 'prodjob_yd5tzc4c9ftzeplzlqyytluich'.
(anyscale +6.4s) View the job in the UI: https://console.anyscale.com/jobs/prodjob_yd5tzc4c9ftzeplzlqyytluich
(anyscale +6.4s) Use `--wait` to wait for the job to run and stream logs.


<img src="https://raw.githubusercontent.com/anyscale/multimodal-ai/refs/heads/main/images/train_job.png" width=1000>

## Evaluation

This tutorial concludes by evaluating the trained model on the test dataset. Evaluation is essentially the same as the batch inference workload where you apply the model on batches of data and then calculate metrics using the predictions versus true labels. Ray Data is hyper optimized for throughput so preserving order isn't a priority. But for evaluation, this approach is crucial. Achieve this approach by preserving the entire row and adding the predicted label as another column to each row.

In [ ]:
from urllib.parse import urlparse
from sklearn.metrics import multilabel_confusion_matrix


In [ ]:
class TorchPredictor:
    def __init__(self, preprocessor, model):
        self.preprocessor = preprocessor
        self.model = model
        self.model.eval()

    def __call__(self, batch, device="cuda"):
        self.model.to(device)
        batch["prediction"] = self.model.predict(collate_fn(batch))
        return batch

    def predict_probabilities(self, batch, device="cuda"):
        self.model.to(device)
        predicted_probabilities = self.model.predict_probabilities(collate_fn(batch))
        batch["probabilities"] = [
            {
                self.preprocessor.label_to_class[i]: float(prob)
                for i, prob in enumerate(probabilities)
            }
            for probabilities in predicted_probabilities
        ]
        return batch
    
    @classmethod
    def from_artifacts_dir(cls, artifacts_dir):
        with open(os.path.join(artifacts_dir, "class_to_label.json"), "r") as fp:
            class_to_label = json.load(fp)
        preprocessor = Preprocessor(class_to_label=class_to_label)
        model = ClassificationModel.load(
            args_fp=os.path.join(artifacts_dir, "args.json"), 
            state_dict_fp=os.path.join(artifacts_dir, "model.pt"),
        )
        return cls(preprocessor=preprocessor, model=model)


In [ ]:
# Load and preproces eval dataset.
artifacts_dir = urlparse(best_run.artifact_uri).path
predictor = TorchPredictor.from_artifacts_dir(artifacts_dir=artifacts_dir)
test_ds = ray.data.read_images("s3://doggos-dataset/test", include_paths=True)
test_ds = test_ds.map(add_class)
test_ds = predictor.preprocessor.transform(ds=test_ds)


In [ ]:
# y_pred (batch inference).
pred_ds = test_ds.map_batches(
    predictor,
    concurrency=4,
    batch_size=64,
    num_gpus=1,
    accelerator_type="L4",
)
pred_ds.take(1)


2025-08-21 20:16:36,704	INFO logging.py:295 -- Registered dataset logger for dataset dataset_54_0
2025-08-21 20:16:36,715	INFO streaming_executor.py:117 -- Starting execution of Dataset dataset_54_0. Full logs are in /tmp/ray/session_2025-08-21_18-48-13_464408_2298/logs/ray-data
2025-08-21 20:16:36,716	INFO streaming_executor.py:118 -- Execution plan of Dataset dataset_54_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[Map(add_class)->Map(convert_to_label)] -> ActorPoolMapOperator[MapBatches(EmbedImages)] -> TaskPoolMapOperator[MapBatches(drop_columns)] -> TaskPoolMapOperator[MapBatches(TorchPredictor)] -> LimitOperator[limit=1]


Running 0: 0.00 row [00:00, ? row/s]

- ListFiles 1: 0.00 row [00:00, ? row/s]

- ReadFiles 2: 0.00 row [00:00, ? row/s]

- Map(add_class)->Map(convert_to_label) 3: 0.00 row [00:00, ? row/s]

...

- MapBatches(TorchPredictor) 6: 0.00 row [00:00, ? row/s]

- limit=1 7: 0.00 row [00:00, ? row/s]

(_MapWorker pid=5340, ip=10.0.164.18) Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
(MapBatches(TorchPredictor) pid=8912, ip=10.0.171.239) /tmp/ipykernel_44250/417303983.py:6: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:203.)
(_MapWorker pid=7119, ip=10.0.171.239) Using a slow image processor as `use_fast` is unset and a slow processor was saved wit

2025-08-21 20:16:56,309	INFO streaming_executor.py:231 -- ✔️  Dataset dataset_54_0 execution finished in 19.59 seconds


[{'path': 'doggos-dataset/test/basset/basset_10005.jpg',
  'class': 'basset',
  'label': 15,
  'embedding': array([ 8.89970511e-02, -5.91553897e-02,  1.15793884e-01,  2.15907171e-01,
         -3.43177855e-01, -3.35305780e-01,  1.48934871e-01, -1.01916254e-01,
         -1.69534028e-01,  4.21351194e-03,  2.40714669e-01,  1.78949311e-01,
          4.34440553e-01,  4.59233075e-01,  1.59075558e-02,  4.16404784e-01,
          5.20997405e-01,  1.86564118e-01, -3.43529493e-01, -4.01162148e-01,
         -1.14708230e-01, -3.04383188e-01,  1.77818760e-01,  1.81996599e-01,
         -3.55898142e-01, -2.30893344e-01,  1.68633550e-01,  3.78260732e-01,
          8.35477859e-02, -4.92601022e-02,  3.17792714e-01, -1.40344769e-01,
         -2.15592146e-01, -4.72647041e-01, -3.01129401e-01,  7.62015060e-02,
         -2.64467388e-01,  5.89425087e-01, -3.75532597e-01,  3.11463177e-01,
         -3.82608265e-01, -1.87145710e-01,  1.07511304e-01,  4.81413454e-01,
         -3.70517075e-01,  9.13751602e-01,  3.1

In [ ]:
def batch_metric(batch):
    labels = batch["label"]
    preds = batch["prediction"]
    mcm = multilabel_confusion_matrix(labels, preds)
    tn, fp, fn, tp = [], [], [], []
    for i in range(mcm.shape[0]):
        tn.append(mcm[i, 0, 0])  # True negatives
        fp.append(mcm[i, 0, 1])  # False positives
        fn.append(mcm[i, 1, 0])  # False negatives
        tp.append(mcm[i, 1, 1])  # True positives
    return {"TN": tn, "FP": fp, "FN": fn, "TP": tp}


In [ ]:
# Aggregated metrics after processing all batches.
metrics_ds = pred_ds.map_batches(batch_metric)
aggregate_metrics = metrics_ds.sum(["TN", "FP", "FN", "TP"])

# Aggregate the confusion matrix components across all batches.
tn = aggregate_metrics["sum(TN)"]
fp = aggregate_metrics["sum(FP)"]
fn = aggregate_metrics["sum(FN)"]
tp = aggregate_metrics["sum(TP)"]

# Calculate metrics.
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
accuracy = (tp + tn) / (tp + tn + fp + fn)


2025-08-21 20:16:56,576	INFO logging.py:295 -- Registered dataset logger for dataset dataset_57_0
2025-08-21 20:16:56,588	INFO streaming_executor.py:117 -- Starting execution of Dataset dataset_57_0. Full logs are in /tmp/ray/session_2025-08-21_18-48-13_464408_2298/logs/ray-data
2025-08-21 20:16:56,589	INFO streaming_executor.py:118 -- Execution plan of Dataset dataset_57_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[Map(add_class)->Map(convert_to_label)] -> ActorPoolMapOperator[MapBatches(EmbedImages)] -> TaskPoolMapOperator[MapBatches(drop_columns)] -> TaskPoolMapOperator[MapBatches(TorchPredictor)] -> TaskPoolMapOperator[MapBatches(batch_metric)] -> AllToAllOperator[Aggregate] -> LimitOperator[limit=1]


Running 0: 0.00 row [00:00, ? row/s]

- ListFiles 1: 0.00 row [00:00, ? row/s]

- ReadFiles 2: 0.00 row [00:00, ? row/s]

- Map(add_class)->Map(convert_to_label) 3: 0.00 row [00:00, ? row/s]

...

Sort Sample 9:   0%|          | 0.00/1.00 [00:00<?, ? row/s]

Shuffle Map 10:   0%|          | 0.00/1.00 [00:00<?, ? row/s]

Shuffle Reduce 11:   0%|          | 0.00/1.00 [00:00<?, ? row/s]

- limit=1 12: 0.00 row [00:00, ? row/s]

2025-08-21 20:18:01,205	INFO streaming_executor.py:231 -- ✔️  Dataset dataset_57_0 execution finished in 64.61 seconds


In [ ]:
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1: {f1:.2f}")
print(f"Accuracy: {accuracy:.2f}")


Precision: 0.84
Recall: 0.84
F1: 0.84
Accuracy: 0.98


**🚨 Note**: Reset this notebook using the **"🔄 Restart"** button location at the notebook's menu bar. This way we can free up all the variables, utils, etc. used in this notebook.